# Kuramoto-Sivashinsky Equation

$$
u_t = -u u_x - u_{xx} - \nu u_{xxxx}  \qquad (0 < x < L)
$$

$$
u_t = -\lambda u u_x - \lambda^2 u_{xx} - \nu \lambda^4 u_{xxxx}, \qquad (0 < x < 1)
$$

In [1]:
%matplotlib qt5

In [2]:
import sys
sys.path.insert(0, '..')

In [3]:
import numpy as np
import equations as eq
from scipy.integrate import odeint
from scipy.fftpack import rfft, irfft

import h5py

C:\Users\ryoikota\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

## Setting 

In [32]:
NW = 48
ks01 = eq.KS(NW)
args = ks01.getParamDefault()

In [33]:
N2 = ks01.N2
J = ks01.J
x = ks01.get_x()

## Setting initial data 

In [34]:
u0 = np.zeros(N2)
u0[1] = 0.5*J

## Computation 

In [35]:
trange = np.linspace(0.0, 1000.0, 501)
dt = 1.0e-2

In [9]:
args

[1.0, 22.0]

In [36]:
with h5py.File('ks.hdf5', 'w') as fh:
    ks01.mkInitDataSet(u0, fh, dt)
    ks01.evolve(fh, trange, tuple(args))
    data = fh['u'][()]

In [37]:
up = ks01.mkPhysData(
        ks01.reshapeTS(data, len(trange)))

## Visualization

In [44]:
fig = plt.figure(figsize=(4,3))
ax = fig.add_subplot(111)

ax.set_ylim((-3.0, 3.0))
ax.set_xlim((-0.1, 1.1))
ax.grid(True)
lines = plt.plot(x, [np.nan]*len(x));

In [45]:
def init():
    for l in lines:
        l.set_ydata([np.nan] * len(x))
    return lines

def animate(i):
    for j, l in enumerate(lines):
        l.set_ydata(up[i,j,:])
    return lines

In [46]:
ani = animation.FuncAnimation(
    fig, animate, frames=len(trange),
    init_func=init, interval=100, blit=True,
    repeat=True)

plt.show()